In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import yfinance as yf
from pandas_datareader import data as pdr 
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm 

In [ ]:
df = pdr.get_data_yahoo('EURUSD=X',start = '2010-01-01')
df['Ret'] = df['Close'].diff()
df['LogClose'] = np.log(df['Close'])
df['LogRet'] = df['LogClose'].diff()

In [ ]:
def adf(x):
    res = adfuller(x)
    print("Test-Statistic:", res[0])
    print("P-Value:", res[1])
    if res [1] < 0.05:
        print("Stationary")
    else:
        print("Non-Stationary")

In [ ]:
EURUSD = df['Close']
Ntest = 12
train = EURUSD.iloc[:-Ntest]
test = EURUSD.iloc[-Ntest:]

In [ ]:
test

In [ ]:
model = pm.auto_arima(train,
                     error_action='ignore',trace=True,
                     suppress_warnings=True, maxiter=10,
                     seasonal=False)

In [ ]:
model.summary()

In [ ]:
def plot_result(model,fulldata,train,test):
    params = model.get_params()
    d = params['order'][1]
    
    train_pred = model.predict_in_sample(start=d, end=-1)
    test_pred, confint = model.predict(n_periods=Ntest, return_conf_int=True)
    
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(fulldata.index,fulldata, label = 'data')
    ax.plot(train.index[d:],train_pred, label = 'fitted')
    ax.plot(test.index, test_pred, label = 'forecast')
    ax.fill_between(test.index, \
                   confint[:,0], confint[:,1], \
                   color='red', alpha=0.3)
    ax.legend();

In [ ]:
plot_result(model,EURUSD, train, test )

In [ ]:
def plot_test(model,test):
    test_pred, confint = model.predict(n_periods=Ntest, return_conf_int=True)
    
    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot(test.index, test, label='data')
    ax.plot(test.index, test_pred, label='forecast')
    ax.fill_between(test.index, \
                   confint[:,0], confint[:,1], \
                   color='red', alpha=0.3)
    ax.legend();

In [ ]:
plot_test(model, test)

In [ ]:
armia = ARIMA(train, order=(1,1,0))

In [ ]:
arima_result = armia.fit()
prediction_result = arima_result.get_forecast(Ntest)
forecast = prediction_result.predicted_mean

In [ ]:
prediction_result.conf_int()

In [ ]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

def plot_fit_and_forecast(result):
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df['Close'], label='data')
    
    #plot the curve fitted on train set 
    train_pred = result.fittedvalues
    ax.plot(train.index, train_pred, color='green', label='fitted')
    
    # forecast the test set 
    prediction_result = result.get_forecast(Ntest)
    conf_int = prediction_result.conf_int()
    lower, upper = conf_int['lower Close'], conf_int['lower Close']
    forecast = prediction_result.predicted_mean 
    ax.plot(test.index, forecast, label='forecast')
    ax.fill_between(test.index, \
                    lower, upper, \
                    color='red', alpha=0.3)
    ax.legend()

In [ ]:
plot_fit_and_forecast(arima_result)

In [ ]:
def plot_forecast(result):
    fig, ax = plt.subplots(figsize=(15,5))
   # ax.plot(df['Close'], label='data')
    
    
    # forecast the test set 
    prediction_result = result.get_forecast(Ntest)
    conf_int = prediction_result.conf_int()
    lower, upper = conf_int['lower Close'], conf_int['lower Close']
    forecast = prediction_result.predicted_mean 
    ax.plot(test.index, forecast, label='forecast')
    ax.fill_between(test.index, \
                    lower, upper, \
                    color='red', alpha=0.3)
    ax.legend()

In [ ]:
plot_forecast(arima_result)